## Emulating Fridrich’s (CFOP) Human Speedcubing Algorithm 

___


This Notebook demonstrates my first successful Rubik's cube solution using an implementation of the (cube-)layer based solution strategy: CFOP. Albeit, the last layer employs an easily searchable set of about 80 explicitly defined move sequences that were developed (primarily) by Jessica Fridrich back in the 80's (see [CFOP_OLL_ALGS.TXT](../data/CFOP_OLL_ALGS.TXT) and [CFOP_PLL_ALGS.TXT](../data/CFOP_PLL_ALGS.TXT) for the explict Singmaster move sequences employed during the OLL and PLL stages).

The first two stages, Cross and F2L, are solved using a stepped series of UCB tree searches. A typical (successful) search will likely generate between 5-15 million nodes in total and visit about a 40-45% of those. It usually takes 5-10 minutes to converge to a solution, though it can take over 20 at times and some cubes fail to converge at all (using my default configuation parameters anyway).

For reference, the lion's share of the solution logic here is contained in the [CfopSolver](../rubiks/solver/CfopSolver.py) and [CfopState](../rubiks/solver/CfopState.py) classes.

___


In [ ]:
import sys

# This for managing relative imports from nb
if '..' not in sys.path: sys.path.append('..')

import numpy as np
from IPython.display import HTML

from rubiks.model.CubeView import CubeView
from rubiks.model.CfopCube import CfopCube
from rubiks.model.SMAdapter import SMAdapter
from rubiks.model.VectorCube import VectorCube, color_name, color_letr

from rubiks.solver.CfopState import F2LStateOrbit
from rubiks.solver.CfopSolver import CfopSolver
from rubiks.solver.FaceletSolver import FaceletSolver

In [ ]:
%%time

# This cell pre-computes a few hash tables used later in heuristics 
# and the CFOP tree-search logic. It will take a few minutes to complete.

fsolver = FaceletSolver()
sequence_hash = fsolver.create_sequence_hash(wildcard=False)
heuristic_hash = fsolver.create_heuristic_hash(sequence_hash)

# Force pre-compute of static 5-deep orbits
oblk = F2LStateOrbit.get_orbit_pairs([38,41])

In [ ]:
# This cell generates starting_cube using a random scramble sequence.
# You can run multiple searches on this same particular scramble by
# simply NOT re-running this cell between searches.

starting_cube = CfopCube()
moves, invmoves = starting_cube.trace_scramble(sz=20, apply_moves=True)

In [ ]:
# This cell initiates the tree search cube: cfop_cube from starting_cube. Re-run this 
# cell to initiate a "fresh" search from the starting_cube permutation, or do NOT
# re-run it to "continue" a search from the cube state achieved on the previous pass. 

cfop_cube = CfopCube(starting_cube)
CfopSolver.ref_moves = invmoves
print("Starting heuristic:", fsolver.heuristic(cfop_cube))
print("Inverted scramble moves:", CfopSolver.ref_moves)
CubeView(cfop_cube).draw_projection()

In [ ]:
%%time

# Default search configs
FIRST_PASS = 50000
SECOND_PASS = 100000

# This kicks off the actual UCB tree search algorithm
solved, moves, solve_state = CfopSolver.cfop_solve(fsolver, cfop_cube, pass1_iters=FIRST_PASS, pass2_iters=SECOND_PASS)

## Animating the Solutions 

___


The next two cells animate both the solution sequence found by the tree search and, subsequently, the trivial but relatively efficient "reverse-scramble" sequence. Both animations employ an alpha mask on any facelet not at its home position. These visualization really demonstrated to me a fundamental distinction between a human solving strategy, that obviously builds up to a solution, versus a more efficient one that must be operating on some other, perhaps entropy-like, ordering principle.

___


In [ ]:
print(f"Tree Search: {len(moves)} moves to solve cube:\n{[f'({color_letr(mv[0])}:{mv[1]})' for mv in moves]}\n")

inner_cube = CfopCube(starting_cube)
adapter = SMAdapter(inner_cube)
view = CubeView(adapter)

view.record_moves()
for mv in moves:
    nhome_mask = np.nonzero(sum(inner_cube.facelet_matrix[2:] != CfopCube._facelet_matrix[2:]) > 0)[0]
    adapter.rotate_local(mv, alpha_masks=nhome_mask)

adapter.rotate_singmaster_seq(['Y2', 'Y2'])
view.record_moves(stop_recording=True)
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())

In [ ]:
print(f"Reverse-scramble: {len(CfopSolver.ref_moves)} moves to solve cube:\n"
      f"{[f'({color_letr(mv[0])}:{mv[1]})' for mv in CfopSolver.ref_moves]}\n")

inner_cube = CfopCube(starting_cube)
adapter = SMAdapter(inner_cube)
view = CubeView(adapter)

view.record_moves()
for mv in CfopSolver.ref_moves:
    nhome_mask = np.nonzero(sum(inner_cube.facelet_matrix[2:] != CfopCube._facelet_matrix[2:]) > 0)[0]
    adapter.rotate_local(mv, alpha_masks=nhome_mask)

adapter.rotate_singmaster_seq(['Y2', 'Y2'])
view.record_moves(stop_recording=True)
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())